In [6]:
from utils.connect import connect
from utils.aed_rows import add_row, edit_row, delete_row
import pandas as pd

In [16]:
# Load credentials (Replace with your credentials file path)
creds = ServiceAccountCredentials.from_json_keyfile_name("credentials.json", scope)
client = gspread.authorize(creds)

In [9]:
# Open the CSV file (Assumes it's in Google Sheets)
# sheet = client.open(os.getenv("sheet")).sheet1  # Access first sheet

# Get data as Pandas DataFrame
data = sh.get_all_records()
df = pd.DataFrame(data)

print(df.head()) 

         Date  Category  Amount Total  Income
0  2025-02-02       fun     2.0  6471       0
1  2025-02-01  haircut     10.0             0
2  2025-02-01   family    100.0             0
3  2025-02-01      rent   261.0             0
4  2025-01-31       fun    14.0             0


In [8]:
sh = connect()

In [30]:
# New row to add
new_row = ["2025-02-02", "fun", 2, "", 0]  # NaN is for Total

# Insert row
sh.insert_row(new_row, 2)

print("Row added successfully!")


Row added successfully!


In [11]:
# Convert 'Date' to datetime format for sorting
df["Date"] = pd.to_datetime(df["Date"])

# Sort by Date (latest first)
df = df.sort_values(by="Date", ascending=False).reset_index(drop=True)

# Convert "Total" column to numeric (handling empty strings)
df["Total"] = pd.to_numeric(df["Total"], errors="coerce")

# Iterate and compute the Total column
for i in range(1, len(df)):  # Start from second row
    if df.loc[i, "Total"] == "" or pd.isna(df.loc[i, "Total"]):  # If Total is empty
        if df.loc[i, "Income"] == 0:  # Expense
            df.loc[i, "Total"] = df.loc[i-1, "Total"] + df.loc[i, "Amount"]
        else:  # Income
            df.loc[i, "Total"] = df.loc[i-1, "Total"] - df.loc[i, "Amount"]

# Convert Total column back to integer
df["Total"] = df["Total"].astype(int)

# Display the result
# import ace_tools as tools
# tools.display_dataframe_to_user(name="Updated Total Calculation", dataframe=df)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 950 entries, 0 to 949
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      950 non-null    object 
 1   Category  950 non-null    object 
 2   Amount    950 non-null    float64
 3   Total     950 non-null    int64  
 4   Income    950 non-null    int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 37.2+ KB


In [20]:
# Convert 'Date' to datetime format (if not already converted)
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

# Now convert back to string format for Google Sheets
df["Date"] = df["Date"].dt.strftime("%Y-%m-%d")

# Convert DataFrame to list of lists (Google Sheets format)
data_to_push = [df.columns.tolist()] + df.values.tolist()

# Overwrite entire sheet with updated data
sh.update("A1", data_to_push)

print("✅ Data successfully updated in Google Sheets!")

/tmp/ipykernel_28458/3795469456.py:11: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sh.update("A1", data_to_push)


✅ Data successfully updated in Google Sheets!


In [22]:
# Open the Google Sheet
sheet = sh

# ✅ 2. Read Data from Google Sheets into a DataFrame
data = sheet.get_all_records()
df = pd.DataFrame(data)

# ✅ 3. Convert 'Date' to datetime format (handling any errors)
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

# ✅ 4. Sort DataFrame by Date (latest first)
df = df.sort_values(by="Date", ascending=False).reset_index(drop=True)

# ✅ 5. Convert "Total" column to numeric (handling empty values)
df["Total"] = pd.to_numeric(df["Total"], errors="coerce")

# ✅ 6. Shift "Total" column down by one row
df["Total"] = df["Total"].shift(1)

# ✅ 7. Set the first row's "Total" to an empty string
df.loc[0, "Total"] = ""

# ✅ 8. Ensure "Total" remains integer where possible
df["Total"] = pd.to_numeric(df["Total"], errors="coerce").astype("Int64")  # Keeps NaN as blank instead of float NaN

# ✅ 9. Convert Date to string format before pushing
df["Date"] = df["Date"].dt.strftime("%Y-%m-%d")

# ✅ 10. Push updated DataFrame back to Google Sheets
data_to_push = [df.columns.tolist()] + df.astype(str).values.tolist()  # Ensure all values are strings for Google Sheets

# Overwrite entire sheet with updated data
sheet.update("A1", data_to_push)

print("✅ Data successfully updated in Google Sheets!")


/tmp/ipykernel_28458/2312491031.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[0, "Total"] = ""
/tmp/ipykernel_28458/2312491031.py:33: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update("A1", data_to_push)


✅ Data successfully updated in Google Sheets!
